# Defining cell types by their electrophysiology

### The brain has thousands of different types of cells. How do we even begin to tease them apart?

We can define neurons by their <b>gene expression patterns</b>, <b>electrophysiology features</b>, and <b>structure</b>. Here, we'll use those three features to compare and contrast cell types in humans and mice.

This notebook will help us investigate specific features in the electrophysiology dataset from the Allen Brain Atlas. See <b>Technical Notes</b> at the end of this notebook for more information about working with the AllenSDK. This notebook is designed to be run after completing <a href="https://ajuavinett.github.io/CellTypesLesson/instructions">this first activity</a>.

### Table of Contents
1. [Step 1. Importing Allen Cell Types Data](#one)
2. [Step 2. Plotting electrophysiology data](#two)
3. [Step 3. Plotting the morphology of the cell](#three)
4. [Step 4. Analyze computed metrics](#four)
5. [Step 5. Comparing human to mouse cells](#five)
<hr>

<a id="one"></a>
## Step 1. Importing Allen Cell Types data
First, we'll `import` the CellTypesCache module. This module provides tools to allow us to get information from the cell types database. 

In order to run the line below, you need to have the AllenSDK installed. You can find information on how to do that locally <a href="http://alleninstitute.github.io/AllenSDK/install.html">here</a>.
- <b>If you're running this on Binder</b>, uncomment line 2 below.
- <b>If you're running this on the UCSD Datahub</b>, the Allen SDK has already been installed for you.

*Note*: There's a lot of code here that we're not explaining. You're not expected to know the ins and outs of this. If you're curious, the CellTypesCache that we're importing provides tools to allow us to get information from the cell types database. We're giving it a **manifest** filename as well. CellTypesCache will create this manifest file, which contains metadata about the cache. If you want, you can look under cell_types in your Jupyter directory, and take a look at the file.

In [ ]:
# If you need to install the AllenSDK, uncomment the line below.
#%pip install allensdk

#Import the "Cell Types Cache" from the AllenSDK core package
from allensdk.core.cell_types_cache import CellTypesCache

#Initialize the cache as 'ctc' (cell types cache)
ctc = CellTypesCache(manifest_file='cell_types/manifest.json')

Now that we have the packages we need, let's import a raw sweep of the data. The cell below will grab the data (in the form of a Neuroscience Without Borders, NWB, file) for the same experiment you just looked at on the website. 

<div class="alert alert-success"><b>Task</b>: Find the cell specimen ID for the first cell you looked at on the Allen website (hint: it's in the URL), and enter this after cell_id below. Run the cell.

This might take a minute or two. You should wait until the circle in the upper right is <i>not</i> filled to continue.</div>

In [ ]:
# Enter your cell_id below
cell_id = 

# Get the electrophysiology (ephys) data for that cell
data = ctc.get_ephys_data(cell_id)
print('Data retrieved')

Thankfully, our NWB file has some built in methods to enable us to pull out a recording sweep. We can access methods of objects like our `data` object by adding a period, and then the method. That's what we're doing below, with `data.get_sweep()`.


<div class="alert alert-success">
    <b>Task:</b> Choose your favorite sweep below. (<u>Hint</u>: go back to the website to see what the sweep numbers are.)</div>
    
<i>Note</i>: You may get an error about something being 'deprecated', but don't worry about it.

In [ ]:
# Assign your favorite sweep number to a variable "sweep_number" below.
sweep_number = 

sweep_data = data.get_sweep(sweep_number) 
print('Sweep obtained')

<a id="two"></a>
## Step 2. Plotting electrophysiology data
Now that you've pulled down some data, chosen a cell, and chosen a sweep number, let's plot that data.

First, let's import and rename a few packages that we need to plot our data.

<div class="alert alert-success">
    <b>Task</b>: Just like you did in the introductory lesson, import the <a href="https://www.numpy.org/">numpy</a> toolbox nicknamed as <code>np</code>. Add a <code>print</code> message at the end that says "Packages imported" so that you know the code ran.</div>

In [ ]:
# Specify that all plots will happen inline
%matplotlib inline  

# Import packages that we need. Put your import line below.
import matplotlib.pyplot as plt

# Import numpy below


# Add your print() statement below


Next, let's plot the sweep we've selected.

<div class="alert alert-success"><b>Task:</b> Run the cell below!</div>

<i>Note</i>: There's a lot of code here, but don't worry about it too much. The principles are the same as in the Introduction to Jupyter Notebooks, except now we're plotting "subplots" -- multiple plots within one image -- so the code is a little bit more complicated.

In [ ]:
# Get the stimulus trace and convert to pA
stim_current = sweep_data['stimulus'] * 1e12 # in A, converted to pA

# Get the voltage trace and convert to mV
response_voltage = sweep_data['response'] * 1e3 # in V, converted to mV

# We need to know the sampling rate so that we can create a time axis for our data
sampling_rate = sweep_data['sampling_rate'] # in Hz
time_stamps = (np.arange(0, len(response_voltage)) * (1.0 / sampling_rate))

# Set up our plot
fig, axes = plt.subplots(2, 1, sharex=True,figsize=(10,10))

# axes 0 is our first plot, of the recorded voltage data
axes[0].plot(time_stamps, response_voltage, color='black')
axes[0].set_ylabel("mV")
#axes[0].set_xlim(0,3)
axes[0].set_title("whole-cell patch recording")

# axes 1 is our second plot, of the stimulus trace
axes[1].plot(time_stamps, stim_current, color='gray')
axes[1].set_ylabel("pA")
axes[1].set_xlabel("seconds")
axes[1].set_title("stimulus")

plt.show()

This plot probably looks different from the plot on the website, why?

<div class="alert alert-success">
<b>Hint:</b> There is a line that has been commented out above. Uncomment the line to change the scaling of the x-axis in order to zoom in on the period where the current was applied. If necessary, change the second value in set_xlim to change the extent of the x axis. Re-run the cell to re-plot the data.</div>

<a id="three"></a>
## Step 3. Plotting the morphology of the cell
The Cell Types Database also contains 3D reconstructions of neuronal morphologies. Here, we'll plot the reconstruction of our cell's morphology. <b>It may take a minute or two to run the cell below.</b>

In [ ]:
# Import necessary toolbox
from allensdk.core.swc import Marker

# Download and open morphology and marker files
morphology = ctc.get_reconstruction(cell_id) 
markers = ctc.get_reconstruction_markers(cell_id) 

# Set up our plot
fig, axes = plt.subplots(1, 2, sharey=True, sharex=True, figsize=(10,10))
axes[0].set_aspect('equal')
axes[1].set_aspect('equal')

# Make a line drawing of x-y and y-z views
for n in morphology.compartment_list:
    for c in morphology.children_of(n):
        axes[0].plot([n['x'], c['x']], [n['y'], c['y']], color='black')
        axes[1].plot([n['z'], c['z']], [n['y'], c['y']], color='black')

# cut dendrite markers
dm = [ m for m in markers if m['name'] == Marker.CUT_DENDRITE ]
axes[0].scatter([m['x'] for m in dm], [m['y'] for m in dm], color='#3333ff')
axes[1].scatter([m['z'] for m in dm], [m['y'] for m in dm], color='#3333ff')

# no reconstruction markers
nm = [ m for m in markers if m['name'] == Marker.NO_RECONSTRUCTION ]
axes[0].scatter([m['x'] for m in nm], [m['y'] for m in nm], color='#333333')
axes[1].scatter([m['z'] for m in nm], [m['y'] for m in nm], color='#333333')
axes[0].set_ylabel('y')
axes[0].set_xlabel('x')
axes[1].set_xlabel('z')
plt.show()

<a id="four"></a>
## Step 4. Analyze computed metrics

The Cell Types Database contains a set of features that have already been computed, which could serve as good starting points for analysis. We can query the database to get these features. Below, we'll use the Pandas package that we imported above to create a **dataframe** for our data.

<div class="alert alert-success">
    <b>Task</b>: Run the cell below. It'll take ~10 seconds. It will print a list of all of the feature available, as well as produce a dataframe, which looks something like an excel spreadsheet. You can scroll to the right to see all of the different features available in this dataset.</div>

<i>Note</i> you may get an error that says 'from_csv is deprecated' while running this, but it won't disrupt the script. You can ignore it.

In [ ]:
import pandas as pd

# Download all electrophysiology features for all cells
ephys_features = ctc.get_ephys_features()
ef_df = pd.DataFrame(ephys_features).set_index('specimen_id')

print('Ephys features available for %d cells:' % len(ef_df))
print(ef_df.columns)
ef_df.head() # Just show the first 5 rows (the head) of our dataframe 

That's how to get all the ephys features for a given specimen - what if we want a particular feature for all cells? Let's first look at the speed of the trough, and the ratio between the upstroke and downstroke of the action potential.
![](https://github.com/ajuavinett/CellTypesLesson/blob/master/docs/ap_features.png?raw=true)
<div style="background: #F3D48D; border-radius: 3px; padding: 10px;">
<b>Action potential fast trough (fast_trough_v_long_square)</b>: Minimum value of the membrane potential in the interval lasting 5 ms after the peak.

<b>Upstroke/downstroke ratio (upstroke_downstroke_ratio_long_square)</b>: The ratio between the absolute values of the action potential peak upstroke and the action potential peak downstroke.</div> 

In [ ]:
plt.scatter(ef_df['fast_trough_v_long_square'], 
            ef_df['upstroke_downstroke_ratio_long_square'])
plt.ylabel("upstroke-downstroke ratio")
plt.xlabel("fast trough depth (mV)")
plt.show()

It looks like there may be roughly two clusters in the data above. Maybe they relate to whether the cells are presumably excitatory (spiny) cells or inhibitory (aspiny) cells. Let's query the API and split up the two sets to see.

<div class="alert alert-success">
<b>Task:</b> The cell below will dig up the dendrite type of these cells and add that to our dataframe. Then, it'll create our same scatterplot, where each dot is colored by dendrite type. All you need to do is run the cell!</div>

In [ ]:
# Get information about our cells' dendrites
cells = ctc.get_cells()
ef_df_dendrites = ef_df.join(pd.DataFrame(cells).set_index('id'))

# Create a dataframe for spiny cells, and a dataframe for aspiny cells
spiny_df = ef_df_dendrites[ef_df_dendrites['dendrite_type'] == 'spiny']
aspiny_df = ef_df_dendrites[ef_df_dendrites['dendrite_type'] == 'aspiny']

# Create our plot! Calling scatter twice like this will draw both of these on the same plot.
plt.scatter(spiny_df['fast_trough_v_long_square'],spiny_df['upstroke_downstroke_ratio_long_square'])
plt.scatter(aspiny_df['fast_trough_v_long_square'],aspiny_df['upstroke_downstroke_ratio_long_square'])

plt.ylabel("upstroke-downstroke ratio")
plt.xlabel("fast trough depth (mV)")
plt.legend(['Spiny','Aspiny'])
    
plt.show()

Looks like these two clusters do partially relate to the dendritic type. Cells with spiny dendrites (which are typically excitatory cells) have a big ratio of upstroke:downstroke, and a more shallow trough (less negative). Cells with aspiny dendrites (typically inhibitory cells) are a little bit more varied. But </i>only</i> aspiny cells have a low upstroke:downstroke ratio and a deeper trough (more negative).

Let's take a closer look at the action potentials of these cells to see what these metrics actually mean for the action potential waveform by choosing one of the cells with the highest upstroke:downstroke ratio.

In [ ]:
# Sort our dataframe so that it's ascending based on upstroke:downstroke ratio
ef_df_upstroke_sorted = ef_df.sort_values('upstroke_downstroke_ratio_long_square',ascending=False)

# Assign one of the top cells in our dataframe and the ratio to different variables
specimen_id = ef_df_upstroke_sorted.index[2]
ratio = ef_df_upstroke_sorted.iloc[2]['upstroke_downstroke_ratio_long_square']

# Print our results so that we can see them
print('Specimen ID: ' + str(specimen_id) + ' with upstroke-downstroke ratio: ' + str(ratio))

Now we can take a closer look at the action potential for that cell by grabbing its electrophysiology data, just like we did above. Let's find a good sweep that will show us some nice action potentials. The next cell of code will look for sweep numbers with a Long Square stimulus of at least 200 pA. It'll print some sweep IDs.

<div class="alert alert-success"><b>Task:</b> Run the cell below!</div>

In [ ]:
# Get the data for our specimen
upstroke_data = ctc.get_ephys_data(specimen_id)

# Get one sweep for our specimen (I've already handselected a gorgeous one for you, 45)
upstroke_sweep = upstroke_data.get_sweep(45) 

# Get the current & voltage traces
current = upstroke_sweep['stimulus'] * 1e12 # in A, converted to pA
voltage = upstroke_sweep['response'] * 1e3 # converted to mV!

# Get the time stamps for our voltage trace
timestamps = (np.arange(0, len(voltage)) * (1.0 / upstroke_sweep['sampling_rate']))

And now let's plot our sweep. You may need to change some values in the plotting script to actually zoom in on the action potential, or change the sweep number to one where you can see an action potential.

<div class="alert alert-success"><b>Task</b>: Modify the plotting script so that you can actually see the shape of the action potential.</div>

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)

# axes 0 is our first plot, of the recorded voltage data
axes[0].plot(timestamps, voltage, color='black')
axes[0].set_ylabel("mV")
axes[0].set_title("whole-cell patch recording")

#axes 1 is our second plot, of the stimulus trace
axes[1].plot(timestamps, current, color='gray')
axes[1].set_ylabel("pA")
axes[1].set_xlabel("seconds")
axes[1].set_title("stimulus")

plt.show()

Let's do the same thing for a cell with a low upstroke ratio and compare. The code below is the same as above, except we've swapped out `True` for `False` where it says `ascending= `. 

<div class="alert alert-success"><b>Task:</b> Run the cells below! Similiar to above, zoom in on the x axis so that you can actually see the shape of the waveform. </div>

In [ ]:
# Sort our dataframe so that it's ascending based on upstroke:downstroke ratio
ef_df_upstroke_sorted = ef_df.sort_values('upstroke_downstroke_ratio_long_square',ascending=True)

# Assign one of the top cells in our dataframe and the ratio to different variables
specimen_id = ef_df_upstroke_sorted.index[2]
ratio = ef_df_upstroke_sorted.iloc[2]['upstroke_downstroke_ratio_long_square']

# Print our results so that we can see them
print('Specimen ID: ' + str(specimen_id) + ' with upstroke-downstroke ratio: ' + str(ratio))

In [ ]:
# Get the data for our specimen
upstroke_data = ctc.get_ephys_data(specimen_id)

# Get one sweep for our specimen (I've already handselected a gorgeous one for you, 45)
upstroke_sweep = upstroke_data.get_sweep(45) 

# Get the current & voltage traces
current = upstroke_sweep['stimulus'] * 1e12 # in A, converted to pA
voltage = upstroke_sweep['response'] * 1e3 # converted to mV!

# Get the time stamps for our voltage trace
timestamps = (np.arange(0, len(voltage)) * (1.0 / upstroke_sweep['sampling_rate']))

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)

axes[0].plot(timestamps, voltage, color='black')
axes[0].set_ylabel("mV")
axes[0].set_title("cell with lowest upstroke:downstroke ratio")

axes[1].plot(timestamps, current, color='gray')
axes[1].set_ylabel("pA")
axes[1].set_xlabel("seconds")
axes[1].set_title("stimulus")

plt.show()

As you can see, even that one metric, upstroke:downstroke ratio, means the shape of the action potential is dramatically different. The other metric above, size of the trough, is highly correlated with upstroke:downstroke. You can see that by comparing the two cells here. Cells with high upstroke:downstroke tend to have less negative troughs (undershoots) after the action potential.

<a id="five"></a>

## Step 5. Comparing human to mouse cells
Let's get out of the action potential weeds a bit. What if we want to know a big picture thing, such as <b>are human cells different than mouse cells?</b>
    
We can make similar plots to above but subselected for human and mouse cells. First, let's get an idea of how many cells of each type are here.

In [ ]:
from allensdk.api.queries.cell_types_api import CellTypesApi

# download all cells
cells = ctc.get_cells()
print("Total cells: %d" % len(cells))
# mouse cells
cells = ctc.get_cells(species=[CellTypesApi.MOUSE])
print("Mouse cells: %d" % len(cells))
# human cells
cells = ctc.get_cells(species=[CellTypesApi.HUMAN])
print("Human cells: %d" % len(cells))

Let's now get all of the electrophysiology data for the mouse and human cells, separately.

In [ ]:
# make a dataframe out of ephys features
ephys_features_df = pd.DataFrame.from_records(ephys_features)
ephys_features_df.head(1)

# grab mouse data and merge with dataframe
mouse_df = pd.DataFrame(ctc.get_cells(species=[CellTypesApi.MOUSE]))
mouse_ephys_df = pd.merge(mouse_df,ephys_features_df,left_on='id',right_on='specimen_id',how='left')

# grab human data and merge with dataframe
human_df = pd.DataFrame(ctc.get_cells(species=[CellTypesApi.HUMAN]))
human_ephys_df = pd.merge(human_df,ephys_features_df,left_on='id',right_on='specimen_id',how='left')

print('Dataframes created.')

Let's look at the first five rows of our mouse and human datasets here.

<div class="alert alert-success"><b>Task</b>: Show the first few rows of the table by using brackets after the variable name. Look above to see which variable your merged dataframe is saved as (hint: it's the variable assigned in the line where <code>pd.merge</code> is used). Then, you'll you want to enter <code>variable_name[1:5]</code>. Do this for the mouse data in the first cell below, and human data in the second. Scroll to the right within the table to see all of the available parameters for the cell.</div>

Let's start by plotting the resting membrane potential for all of our human cells vs all of our mouse cells.

In [ ]:
plt.figure()

# Change your parameter below. Make sure this matches the names in the list you generated above!
parameter = 'vrest'

# Below it is set to plot resting membrane potential 'vrest'.
# That's where you can change what is being plotted.
# Make sure you change it for both the mouse and human plot!
plt.hist([mouse_ephys_df[parameter],human_ephys_df[parameter]],color=[(0, .5,.5, 0.5),(0, 0, 1, 0.5)])

# this is where you should change the label:
plt.xlabel('Resting Membrane Potential (mV)')
plt.ylabel('Number of Cells')
plt.legend(['mouse','human'])
plt.show()

<div class="alert alert-success"> <b>Task</b>: Choose a different parameter to compare between human and mouse cells, and rerun the plot above. Use the documentation below to get the exact name of the parameter (in parentheses), and change the x label axis so that we know what you're plotting.

Save your figure by right clicking on the image above. You'll need to submit this with the assignment.</div>

Here are a few additional metrics you might consider comparing (you can find a complete glossary [here](https://docs.google.com/document/d/1YGLwkMTebwrXd_1E817LFbztMjSTCWh83Mlp3_3ZMEo/edit?usp=sharing):

<div style="background: #F3D48D; border-radius: 3px; padding: 10px;">
<b>Action potential fast trough (fast_trough_v_long_square)</b>: Minimum value of the membrane potential in the interval lasting 5 ms after the peak.
    
<b>Upstroke/downstroke ratio (upstroke_downstroke_ratio_long_square)</b>: The ratio between the absolute values of the action potential peak upstroke and the action potential peak downstroke.
    
<b>Adapation ratio (adaptation)</b>: The rate at which firing speeds up or slows down during a stimulus<br>
    
<b>Average ISI (avg_isi)</b>: The mean value of all interspike intervals in a sweep.<br>
    
<b>Voltage of after-hyperpolarization (trough_v_short_square)</b>: minimum value of the membrane potential during the after-hyperpolarization</div>
    

### Subselect for spiny or aspiny cells.
Final step for today! The histogram above is for <i>all cell types</i>, which is a really hetergenous bunch.

What happens if we subselect our human and mouse cells to be just spiny, or just aspiny? Will those types be more different in humans and mice?

<div class="alert alert-success"><b>Task</b>: Decide whether you want to look at spiny or aspiny cells by setting cell_type to either 'spiny' or 'aspiny' below. The value of your variable needs to be a <b>string</b> -- in other words, it should have quotes around it.</div>

In [ ]:
cell_type =

human = human_ephys_df[human_ephys_df['dendrite_type']== cell_type]
mouse = mouse_ephys_df[mouse_ephys_df['dendrite_type']== cell_type]
print(['Number of human' + cell_type + 'cells: %d' % len(human)])
print(['Number of mouse' + cell_type + 'cells: %d' % len(mouse))

# Change your parameter below.
parameter = 'vrest'

plt.hist([mouse[parameter],human[parameter]],color=[(0, .5,.5, 0.5),(0, 0, 1, 0.5)])
plt.xlabel('Resting Membrane Potential (mV)')
plt.ylabel('Number of Cells')
plt.legend(['mouse','human'])

#cells_concat = pd.concat([mouse[parameter],human[parameter]],axis=1)
#cells_concat.columns = ['mouse','human']
#cells_concat.boxplot()
#plt.ylabel('Resting Membrane Potential (mv)')

plt.show()

<div class="alert alert-success"><b>Task</b>: It's a little hard to see differences here, so let's plot it as a boxplot. <b>Comment</b> out the lines for the histogram (add a <code>#</code> and <b><i>uncomment</b></i> the four lines below. Re-run the code to get a boxplot of the data.

Change the metric here to whatever metric you created and saved a histogram for above. Save your image when you're done.

Submit both of your images along with your answers to the worksheet.
</div>

In [ ]:
from IPython.display import HTML
print('Nice work!')
HTML('<img src="https://media.giphy.com/media/xUOwGhOrYP0jP6iAy4/giphy.gif">')

-----------

#### Technical notes & credits

This notebook demonstrates most of the features of the AllenSDK that help manipulate data in the Cell Types Database.  The main entry point will be through the `CellTypesCache` class. `CellTypesCache` is responsible for downloading Cell Types Database data to a standard directory structure on your hard drive.  If you use this class, you will not have to keep track of where your data lives, other than a root directory.

Much more information can be found in the <a href="http://help.brain-map.org/download/attachments/8323525/CellTypes_Ephys_Overview.pdf">Allen Brain Atlas whitepaper</a> as well as in their <a href="http://alleninstitute.github.io/AllenSDK/cell_types.html">GitHub documentation</a>.

This file modified from <a href='https://alleninstitute.github.io/AllenSDK/_static/examples/nb/cell_types.html'>this</a> notebook.

In case you're curious, <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.plot.html ">here's documentation</a> for plotting pandas series (which we do quite a bit above). You can always Google questions you have!)